In [6]:
# SageMath 脚本
def solve():
    # 1. 定义常数
    M = 2^64
    A = 2862933555777941757
    C = 3037000493
    
    # 题目给出的观测值 (高32位)
    glimpses = [1332353447, 4117841481, 4217378265]
    
    # 2. 准备构建格所需的参数
    # 计算 K_i (Bias)
    # 关系推导: x_i = A^i * x_0 + K_i (mod M)
    # K_i = (A^i * y0_full + Cumulative_C_i - yi_full) % M
    
    K = []
    
    # 计算 A^i 和 累积 C
    # S_i = A^i * S_0 + term_c
    # term_c[0] = 0
    # term_c[1] = C
    # term_c[2] = A*C + C
    
    term_c = [0, C, (A*C + C) % M]
    term_a = [1, A, (A^2) % M]
    
    base_shift = 2^32
    y0_full = glimpses[0] * base_shift
    
    # 计算 K1, K2 (对应 glimpse[1], glimpse[2])
    # 注意：对于 i=0, x0 = 1*x0 + 0, 所以 K0=0，我们在矩阵中不需要 K0，只处理 K1, K2
    for i in range(1, 3):
        yi_full = glimpses[i] * base_shift
        # K_i = A^i * (y0_full) + term_c[i] - yi_full
        val = (term_a[i] * y0_full + term_c[i] - yi_full) % M
        K.append(val)
        
    # 3. 构建矩阵
    # 目标寻找短向量: (x0, x1, x2, 1)
    # Matrix B (4x4):
    # [ 1,    A,    A^2,  0 ]
    # [ 0,    M,    0,    0 ]
    # [ 0,    0,    M,    0 ]
    # [ 0,    K1,   K2,   1 ]
    
    B = Matrix(ZZ, 4, 4)
    B[0, 0] = 1
    B[0, 1] = term_a[1]
    B[0, 2] = term_a[2]
    B[0, 3] = 0
    
    B[1, 1] = M
    B[2, 2] = M
    
    B[3, 1] = K[0] # K1
    B[3, 2] = K[1] # K2
    B[3, 3] = 1    # Constant constrol
    
    print("Executing LLL reduction...")
    L = B.LLL()
    
    # 4. 从规约后的基中提取解
    real_x0 = None
    
    for row in L:
        # LLL 可能产生正负号翻转的向量，检查最后一维是 1 或 -1
        vec = list(row)
        if abs(vec[3]) == 1:
            potential_x0 = vec[0] * vec[3] # 修正符号
            
            # 验证 x0 范围
            if 0 <= potential_x0 < 2^32:
                # 进一步验证是否符合 glimpse 序列
                s0 = y0_full + potential_x0
                
                # 模拟推导
                curr = s0
                valid = True
                for g in glimpses:
                    if (curr >> 32) != g:
                        valid = False
                        break
                    curr = (curr * A + C) % M
                
                if valid:
                    real_x0 = potential_x0
                    print(f"\n[+] Found x0: {real_x0}")
                    print(f"[+] Recovered Initial State S0: {s0}")
                    break
    
    if real_x0 is not None:
        # 5. 预测后5个状态
        # 当前我们验证完 glimpses 后，curr 已经是 S3 (即第3个输出之后的下一个状态)
        # 题目要求预测接下来 5 个
        
        state = (y0_full + real_x0)
        
        # 先空转前3次 (对应已知的3个输出)
        for _ in range(3):
            state = (state * A + C) % M
            
        print("\n[+] Next 5 full 64-bit states:")
        results = []
        for i in range(5):
            print(f"State {i+1}: {state}")
            results.append(str(state))
            state = (state * A + C) % M
            
        print("\nFinal Answer String:")
        print(" ".join(results))
    else:
        print("[-] LLL failed.")

solve()

Executing LLL reduction...

[+] Found x0: 1162129244
[+] Recovered Initial State S0: 5722414482739998556

[+] Next 5 full 64-bit states:
State 1: 172645297224485511
State 2: 7633516833898572440
State 3: 5969584701298477925
State 4: 12291851195862230526
State 5: 13161189394887810867

Final Answer String:
172645297224485511 7633516833898572440 5969584701298477925 12291851195862230526 13161189394887810867
